In [64]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()


20-01-13:03:19:28 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-01-13:03:19:33 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmph67d822p.yml
20-01-13:03:19:34 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [65]:
clipper_conn.register_application(name="sums", input_type="doubles", default_output="-1.0", slo_micros=1000000)

20-01-13:03:19:37 INFO     [clipper_admin.py:236] [default-cluster] Application sums was successfully registered


In [66]:
clipper_conn.get_all_apps()

['sums']

In [67]:
def sumOfList(list1):
    total = 0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return str(total)

In [68]:
from clipper_admin.deployers import python as python_deployer

In [69]:
python_deployer.deploy_python_closure(clipper_conn, name="sums-list", version=1, input_type="doubles", func=sumOfList)

20-01-13:03:19:48 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp1os8xgx2clipper
20-01-13:03:19:48 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-01-13:03:19:48 INFO     [python.py:192] Python closure saved
20-01-13:03:19:48 INFO     [python.py:206] Using Python 3.6 base image
20-01-13:03:19:48 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp1os8xgx2clipper
20-01-13:03:19:48 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-01-13:03:19:48 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-01-13:03:19:48 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmp1os8xgx2clipper /model/
20-01-13:03:19:48 INFO     [clipper_admin.py:539] [default-cluster]  ---> ad1e4b27785b
20-01-13:03:19:48 INFO     [clipper_admin.py:539] [default-cluster] Successfully built ad1e4b27785b
20-01-13:03:19:48 INFO   

In [70]:
clipper_conn.link_model_to_app(app_name="sums", model_name="sums-list")

20-01-13:03:20:02 INFO     [clipper_admin.py:303] [default-cluster] Model sums-list is now linked to application sums


In [71]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/sums/predict", headers=headers, data=json.dumps({"input": list(np.random.random(10))})).json()

{'query_id': 0,
 'output': -1.0,
 'default': True,
 'default_explanation': 'Failed to retrieve a prediction response within the specified latency SLO'}

In [73]:
clipper_conn.stop_all()

20-01-13:06:40:19 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers


为什么 总是默认输出 slo已经加到 两个小时还是 这个 输出  觉得 是函数问题 但函数在 pycharm运行正常 